#0. Install packages

In [ ]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 6.0 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1232 sha256=1f4e038bab878f47ca25dbf084eb26f1025fdf7b33a96720a045ae270c4c32fc
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  1644k      0 --:--:-- --:--:-- --:--:-- 1644k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

In [ ]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3998, done.
remote: Counting objects: 100% (1026/1026), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 3998 (delta 890), reused 859 (delta 826), pack-reused 2972
Receiving objects: 100% (3998/3998), 8.30 MiB | 26.47 MiB/s, done.
Resolving deltas: 100% (2528/2528), done.
/content/fastText
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++17 -march=na

# 1. Load data

In [ ]:
import re  # 정규 표현식 라이브러리
import pandas as pd
import urllib.request  # URL 요청을 위한 라이브러리
from tqdm import tqdm  # 진행률 표시 막대 라이브러리
import hgtk  # 한글 자모 분리 및 결합 라이브러리

In [ ]:
# 네이버 쇼핑 리뷰
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7f82e4463730>)

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


In [ ]:
total_data.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


# 2. hgtk tutorial

- word embedding이 단어 단위의 임베딩이었다면, character embedding은 문자 단위의 임베딩
- 한국어를 character embedding할 수 있는 것이 자음 모음 분리기 hgtk

- 영어는 하나의 알파벳(52자)를 기준으로 character embedding을 하지만, 한국어에서 하나의 음절별로 character embedding을 하면 11172개의 음절이 있기 때문에 계산량이 너무 많다. 따라서 그보다 작은 단위인 자음,모음으로 분리하는 것

 >참고 repo: https://github.com/bluedisk/hangul-toolkit

In [ ]:
# 한글인지 체크
print(hgtk.checker.is_hangul('ㄱ'))
print(hgtk.checker.is_hangul('12'))
print(hgtk.checker.is_hangul('a'))

True
False
False


In [ ]:
# 음절을 초성, 중성, 종성으로 분해
print(hgtk.letter.decompose('남'))
# 초성, 중성, 종성을 하나의 음절로 결합
print(hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ'))

('ㄴ', 'ㅏ', 'ㅁ')
남


In [ ]:
# 결합할 수 없는 상황에서는 에러 발생
try:
  hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ') # 중성이 없는 경우
except:
  print('에러 발생')

에러 발생


# 3. Data Preprocessing

In [ ]:
def word_to_jamo(token):
  def to_special_token(jamo): # 경우에 따라 초, 중, 종성이 다 있는 게 아닌 경우도 있다. 이 경우 -를 반환해주는 함수
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [ ]:
print(word_to_jamo('남동생'))
print(word_to_jamo('야구')) # 야구의 경우 종성이 없으므로 종성 부분을 -로 반환

ㄴㅏㅁㄷㅗㅇㅅㅐㅇ
ㅇㅑ-ㄱㅜ-


In [ ]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [ ]:
# mecab으로 형태소를 분리해주고 그 형태소마다 각각 자음모음을 분리해주는 함수
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [ ]:
# 리뷰 데이터의 reviews 컬럼만을 가져와서 자모 분리
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_numpy()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

100%|██████████| 200000/200000 [01:01<00:00, 3263.90it/s]


In [ ]:
print(len(tokenized_data))
print("전처리 전:", total_data['reviews'][1])
print("전처리 후:", tokenized_data[1])

200000
전처리 전: 택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
전처리 후: ['ㅌㅐㄱㅂㅐ-', 'ㄱㅏ-', 'ㅇㅓㅇㅁㅏㅇ', 'ㅇㅣ-', 'ㄴㅔ-', 'ㅇㅛㅇ', 'ㅈㅓ-ㅎㅢ-', 'ㅈㅣㅂ', 'ㅁㅣㅌ', 'ㅇㅔ-', 'ㅊㅡㅇ', 'ㅇㅔ-', 'ㅁㅏㄹ', 'ㄷㅗ-', 'ㅇㅓㅄㅇㅣ-', 'ㄴㅘ-ㄷㅜ-', 'ㄱㅗ-', 'ㄱㅏ-', 'ㄱㅗ-']


단어를 자모 분리한 것을 역으로 하여 자모 상태를 단어로 다시 결합시키는 함수도 정의해봅시다. 이는 단어의 코사인 유사도를 평가할 때 자모 분리가 된 상태가 아니라 단어 상태로 편리하게 보기 위함입니다.

In [ ]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

#4. FastText

In [ ]:
import fasttext

fasttext를 실행하기에 앞서 훈련 대상인 단어들을 txt 파일로 준비해둬야 합니다. 따라서 `tokenized_data.txt`라는 파일을 쓰기 모드(w)로 생성해주고 앞서 전처리한 `tokenized_data`를 입력해줍니다.

In [ ]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 205314.64 line/s]


아래처럼 `train_unsupeviesd` 함수는 훈련을 시켜주는 함수입니다. 인자로 훈련할 단어가 담긴 txt 파일을 지정하고 model을 `cbow`나 `skipgram` 중에 하나를 고르면 됩니다.

In [ ]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [ ]:
model.save_model("fasttext.bin")

In [ ]:
model = fasttext.load_model("fasttext.bin")

In [ ]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

array([ 2.85749108e-01,  5.32697797e-01,  9.72192764e-01, -1.74561024e-01,
       -9.40615535e-01, -8.52213144e-01,  1.84142113e-01,  7.23624051e-01,
       -6.02583587e-01, -4.90640759e-01,  9.03603256e-01,  2.96650290e-01,
       -2.10492730e-01,  6.45978987e-01, -5.46604753e-01,  6.41264975e-01,
        6.83590710e-01,  5.07914782e-01,  1.90157950e-01, -3.27531621e-02,
        5.80425084e-01, -5.27899086e-01,  6.99647903e-01, -1.41876325e-01,
        5.80996238e-02, -3.13640386e-01,  7.06844479e-02,  1.19922531e+00,
        9.48624730e-01, -7.08683491e-01,  5.12313426e-01, -8.10058236e-01,
        2.31832623e-01, -2.90871948e-01,  1.51137781e+00, -2.15766624e-01,
        4.38416228e-02, -2.49742463e-01,  9.85836610e-02,  1.48401380e-01,
        4.82708663e-01, -9.81113911e-02,  3.92483830e-01, -8.28986242e-02,
       -2.54946172e-01, -1.10600853e+00,  7.52483681e-03,  3.19441855e-01,
       -6.56547397e-02, -2.13221177e-01, -2.11511150e-01, -2.59903312e-01,
        1.69138134e-01,  

`get_nearest_neighbors` 함수를 사용하여 '남동생'이라는 단어와 가장 유사도가 높은 단어들(자모 분리된 상태)을 k개만큼 출력해줍니다.

In [ ]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8671373724937439, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8345811367034912, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7394193410873413, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7316157817840576, 'ㅊㅣㄴㄱㅜ-'),
 (0.7173355221748352, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7168329358100891, 'ㄴㅏㅁㅇㅏ-'),
 (0.7005258202552795, 'ㅈㅗ-ㅋㅏ-'),
 (0.6888477802276611, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.6667895317077637, 'ㅇㅓㄴㄴㅣ-'),
 (0.6643229126930237, 'ㄴㅏㅁㅈㅏ-')]

In [ ]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('구매'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('배달'), k=10)))

[('동생', 0.8671373724937439), ('남친', 0.8345811367034912), ('남편', 0.7394193410873413), ('친구', 0.7316157817840576), ('생일', 0.7173355221748352), ('남아', 0.7168329358100891), ('조카', 0.7005258202552795), ('중학생', 0.6888477802276611), ('언니', 0.6667895317077637), ('남자', 0.6643229126930237)]
[('구매처', 0.8485594987869263), ('구입', 0.8151829242706299), ('주문', 0.7182267308235168), ('주문건', 0.6859400868415833), ('구매자', 0.6213594675064087), ('주문서', 0.6170161962509155), ('구메', 0.6099585890769958), ('헤매', 0.5940358638763428), ('구명조끼', 0.5845101475715637), ('구이', 0.5773112177848816)]
[('배송지', 0.796425461769104), ('깨달', 0.7708380818367004), ('매달', 0.7456046342849731), ('운송장', 0.7196446657180786), ('출고가', 0.7136244177818298), ('택배', 0.7071263194084167), ('메달', 0.7013647556304932), ('배소', 0.6943267583847046), ('송장', 0.6902399063110352), ('직송', 0.688640296459198)]


#5. Word2Vec

In [ ]:
# 간단하게 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

tokenized_data2 = []
for sentence in tqdm(total_data['reviews'].to_list()):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data2.append(stopwords_removed_sentence)

100%|██████████| 200000/200000 [00:24<00:00, 8122.92it/s]


In [ ]:
print("word2vec용 데이터:", tokenized_data2[0])
print("fasttext용 데이터:", tokenized_data[0])

word2vec용 데이터: ['배공', '빠르', '고', '굿']
fasttext용 데이터: ['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']


In [ ]:
from gensim.models import Word2Vec

model2 = Word2Vec(sentences = tokenized_data2, vector_size = 1000, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
# 단어의 총 개수는 14959개이고 벡터 차원은 1000으로 축소되었다.
model2.wv.vectors.shape

(15005, 1000)

#6. FastText와 Word2Vec 결과 비교

### **Q1) 남동생과 주문 두 단어를 input으로 넣고 결과를 비교한 뒤 해석해보세요**

*(Hint: 유사도, 단어의 의미, 단어의 생김새 등을 고려해 볼 수 있습니다)*

In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("남동생"))

FastText 유사도: [('동생', 0.8671373724937439), ('남친', 0.8345811367034912), ('남편', 0.7394193410873413), ('친구', 0.7316157817840576), ('생일', 0.7173355221748352), ('남아', 0.7168329358100891), ('조카', 0.7005258202552795), ('중학생', 0.6888477802276611), ('언니', 0.6667895317077637), ('남자', 0.6643229126930237)]
Word2Vec 유사도: [('사촌', 0.7192986011505127), ('중학생', 0.7174418568611145), ('여친', 0.7095065116882324), ('편찮', 0.6990836262702942), ('입학', 0.6977506279945374), ('개업', 0.6952235102653503), ('래미', 0.6938014030456543), ('어린이날', 0.6921955347061157), ('시어머님', 0.6906082034111023), ('언니', 0.6769683361053467)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('주문'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("주문"))

FastText 유사도: [('주문건', 0.906598687171936), ('주문서', 0.8494508266448975), ('주문자', 0.7549388408660889), ('주무시', 0.740752637386322), ('주무', 0.7267564535140991), ('구입', 0.7227046489715576), ('구매', 0.718227207660675), ('시켰었', 0.6814103126525879), ('구매처', 0.6687557101249695), ('시켰', 0.6650528907775879)]
Word2Vec 유사도: [('구매', 0.8343599438667297), ('구입', 0.8167253136634827), ('선택', 0.6483748555183411), ('장만', 0.5884487628936768), ('시켰', 0.5500988960266113), ('결제', 0.5334495902061462), ('신청', 0.5297647714614868), ('도전', 0.52190762758255), ('도착', 0.5202282667160034), ('시킨', 0.5089359283447266)]


- FastText는 단어를 여러 하위 단어로 나누어 벡터화하기 때문에, 철자 유사성을 잘 포착한다. 예를 들어, '동생', '남친', '남편' 등의 단어가 높은 유사도를 보이는 이유는 이들이 철자상 유사하고 가족 및 친밀한 관계를 의미하는 하위 단어를 공유하기 때문이다.

- Word2Vec은 전체 단어를 단위로 임베딩을 생성한다. 이로 인해 단어 간의 문맥적 유사성을 잘 포착한다.'사촌', '중학생', '여친' 등 단어뿐만 아니라 '편찮', '입학' 등의 특정 상황과 관련된 단어도 포함되어 있는 것을 볼 때, 문맥적으로 넓은 범위의 유사성을 반영하고 있다.

### **Q2) Fasttext가 Word2Vec보다 항상 성능이 나은지 다양한 input을 넣어서 시도해보세요**

In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('구매'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar('구매'))

FastText 유사도: [('구매처', 0.8485594987869263), ('구입', 0.8151829242706299), ('주문', 0.7182267308235168), ('주문건', 0.6859400868415833), ('구매자', 0.6213594675064087), ('주문서', 0.6170161962509155), ('구메', 0.6099585890769958), ('헤매', 0.5940358638763428), ('구명조끼', 0.5845101475715637), ('구이', 0.5773112177848816)]
Word2Vec 유사도: [('구입', 0.8751450181007385), ('주문', 0.8343600034713745), ('장만', 0.590076208114624), ('선택', 0.5414572358131409), ('도전', 0.5350730419158936), ('이용', 0.5220390558242798), ('정착', 0.5207691192626953), ('애용', 0.48348355293273926), ('준비', 0.4831966161727905), ('교체', 0.47693657875061035)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('배달'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar('배달'))

FastText 유사도: [('배송지', 0.796425461769104), ('깨달', 0.7708380818367004), ('매달', 0.7456046342849731), ('운송장', 0.7196446657180786), ('출고가', 0.7136244177818298), ('택배', 0.7071263194084167), ('메달', 0.7013647556304932), ('배소', 0.6943267583847046), ('송장', 0.6902399063110352), ('직송', 0.688640296459198)]
Word2Vec 유사도: [('발송', 0.767374575138092), ('출고', 0.7305788397789001), ('배송', 0.693849503993988), ('택배', 0.6564017534255981), ('누락', 0.6477853655815125), ('수령', 0.6444128751754761), ('분실', 0.637484073638916), ('문자', 0.6316843032836914), ('메일', 0.6144695281982422), ('대처', 0.6137033700942993)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('선물'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar('선물'))

FastText 유사도: [('침전물', 0.765680730342865), ('생일', 0.7224217653274536), ('건물', 0.7050724625587463), ('동생', 0.6862878799438477), ('조카', 0.683184027671814), ('준비물', 0.677862286567688), ('친구', 0.6675467491149902), ('민물', 0.6636090278625488), ('지인', 0.6539587378501892), ('남동생', 0.6538267135620117)]
Word2Vec 유사도: [('소개', 0.6252402067184448), ('이벤트', 0.5745997428894043), ('조카', 0.5505919456481934), ('친구', 0.5447545647621155), ('답례품', 0.5396566987037659), ('동생', 0.5346577167510986), ('지인', 0.5291219353675842), ('나눠', 0.5159536600112915), ('손님', 0.5096608996391296), ('엄마', 0.5052932500839233)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('엄마'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar('엄마'))

FastText 유사도: [('어머님께', 0.8143840432167053), ('아빠', 0.807851254940033), ('어머님', 0.8049381375312805), ('어머니', 0.7885432243347168), ('어머', 0.7670978903770447), ('친정아버지', 0.7554815411567688), ('아버님', 0.7538725733757019), ('시어머님', 0.7388752698898315), ('맘마', 0.7316882014274597), ('아버지', 0.7263551354408264)]
Word2Vec 유사도: [('어머니', 0.8698621988296509), ('아빠', 0.8638588786125183), ('아버지', 0.8370184898376465), ('친정', 0.8235096335411072), ('언니', 0.7507498860359192), ('시어머니', 0.7459112405776978), ('할머니', 0.7445608973503113), ('어머님', 0.7373915910720825), ('시댁', 0.7311649322509766), ('동생', 0.7298002243041992)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('생일'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar('생일'))

FastText 유사도: [('준비물', 0.7789905667304993), ('어린이날', 0.7460465431213379), ('동생', 0.7289976477622986), ('선물', 0.7224217653274536), ('남동생', 0.7173352837562561), ('어버이날', 0.7155279517173767), ('준비', 0.6957136988639832), ('기념일', 0.693120002746582), ('평일', 0.6871252059936523), ('목요일', 0.6697961091995239)]
Word2Vec 유사도: [('크리스마스', 0.8422470688819885), ('어린이날', 0.8129876852035522), ('어버이날', 0.8121942281723022), ('어린이집', 0.803057074546814), ('개업', 0.7926291823387146), ('답례품', 0.7728542685508728), ('생신', 0.772401750087738), ('조카', 0.7562519907951355), ('집들이', 0.7453169226646423), ('동생', 0.7331491112709045)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('기념일'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar('기념일'))

FastText 유사도: [('기일', 0.8980127573013306), ('기념', 0.8095451593399048), ('내일', 0.7703114748001099), ('휴일', 0.7263966798782349), ('독일', 0.7114870548248291), ('평일', 0.705859363079071), ('공휴일', 0.7004603743553162), ('시일', 0.6963574886322021), ('통일', 0.6944978833198547), ('생일', 0.6931201219558716)]
Word2Vec 유사도: [('졸업', 0.7618574500083923), ('친한', 0.7467847466468811), ('앞둔', 0.741023063659668), ('트롤리', 0.7330898642539978), ('큰아이', 0.7235726714134216), ('◡', 0.7204511165618896), ('어린이날', 0.7199611663818359), ('접대용', 0.718529462814331), ('사촌', 0.7173088788986206), ('대성공', 0.7157454490661621)]


- Fasttext가 Word2Vec보다 항상 성능이 낫지는 않다.
- 예를 들어 생일, 기념일 등의 단어 유사도의 경우 Word2Vec이 상대적으로 우수한 것을 확인할 수 있다.